The UCSC mapping seems to be missing some combinations of characters. It looks like the mapping has been trimmed to cater ont to widely used combinations. eg. "බෛ" is missing in the set for "ෛ". At the time of the development of the tool, computation capacity may have been a factor in trimming the mappings.

However, focus of this tool is to cast a reasonably wide net to capture all characters. Therefore we are generating the mappings to capture all available combinations.

Generation will adhere to the following rules:
* New mappings should include all the mappings in UCSC mappings.
* No new mapping should conflict with old mappings

In [1]:
consonants = {
    "l": "ක",
    "L": "ඛ",
    ".": "ග",
    ">": "ඝ",
    "X": "ඞ",
    "Õ": "ඟ",
    "p": "ච",
    "P": "ඡ",
    "c": "ජ",
    "CO": "ඣ",
    "®": "ඣ",
    "[": "ඤ",
    "{": "ඥ",
    "g": "ට",
    "G": "ඨ",
    "v": "ඩ",
    "V": "ඪ",
    "K": "ණ",
    "~": "ඬ",
    ";": "ත",
    ":": "ථ",
    "o": "ද",
    "O": "ධ",
    "k": "න",
    "|": "ඳ",
    "m": "ප",
    "M": "ඵ",
    "n": "බ",
    "N": "භ",
    "u": "ම",
    "U": "ඹ",
    "h": "ය",
    "r": "ර",
    ",": "ල",
    "j": "ව",
    "Y": "ශ",
    "I": "ෂ",
    "i": "ස",
    "y": "හ",
    "<": "ළ",
    "¿": "ළු",
    "*": "ෆ",
}
additional_consonants = {
    "CI": "ක්‍ෂ",
    "Cj": "ක්‍ව",
    "Ë": "ක්‍ෂ",
    "†": "ත්‍ථ",
    "…": "ත්‍ව",
    "‡": "න්‍ද",
    "JO": "න්‍ධ",
}
extra_mappings = {"`.": "ඟ", "`P": "ඦ", "`v": "ඬ", "`o": "ඳ"}
other_letters = {"„": "ද්‍ව", "`j": "ද්‍ව", "Š": "ද්‍ධ", "`O": "ද්‍ධ", "`G": "ට්‍ඨ"}
vowels = {
    "w": "අ",
    "wd": "ආ",
    "we": "ඇ",
    "wE": "ඈ",
    "b": "ඉ",
    "B": "ඊ",
    "W": "උ",
    "W!": "ඌ",
    "R": "ඍ",
    "RD": "ඎ",
    "Ì": "ඏ",
    "Ï": "ඐ",
    "t": "එ",
    "ta": "ඒ",
    "ft": "ඓ",
    "T": "ඔ",
    "´": "ඕ",
    "T!": "ඖ",
}

In [2]:
def concat(fm_prefix, fm_suffix, unicode_prefix, unicode_suffix, in_map, out_map):
    for consonant in in_map.items():
        out_map[fm_prefix + consonant[0] + fm_suffix] = (
            unicode_prefix + consonant[1] + unicode_suffix
        )


def generator(fm_prefix, fm_suffix, unicode_prefix, unicode_suffix, additional=False):

    out = {}

    concat(fm_prefix, fm_suffix, unicode_prefix, unicode_suffix, consonants, out)
    concat(fm_prefix, fm_suffix, unicode_prefix, unicode_suffix, extra_mappings, out)
    if additional:
        concat(
            fm_prefix,
            fm_suffix,
            unicode_prefix,
            unicode_suffix,
            additional_consonants,
            out,
        )
    return out

In [3]:
def printer(char_set):
    print("{", end="")
    for item in char_set.items():
        print(f"'{item[0]}': '{item[1]}'", end=", ")
    print("}", end="")

Mapping for "ෛ" seems to be missing several consonants. Due to the nature of unicode spec, modifiers needs to come after the letter. So this won't be correctly rendered for any case in the form `ff<letter>` if there is no mapping to replace it with the correct unicode form. Therefore generating the combinations and adding them.

In [4]:
char_set_list = []
set_1 = generator("ff", "", "", "ෛ")

print(set_1)

{'ffl': 'කෛ', 'ffL': 'ඛෛ', 'ff.': 'ගෛ', 'ff>': 'ඝෛ', 'ffX': 'ඞෛ', 'ffÕ': 'ඟෛ', 'ffp': 'චෛ', 'ffP': 'ඡෛ', 'ffc': 'ජෛ', 'ffCO': 'ඣෛ', 'ff®': 'ඣෛ', 'ff[': 'ඤෛ', 'ff{': 'ඥෛ', 'ffg': 'ටෛ', 'ffG': 'ඨෛ', 'ffv': 'ඩෛ', 'ffV': 'ඪෛ', 'ffK': 'ණෛ', 'ff~': 'ඬෛ', 'ff;': 'තෛ', 'ff:': 'ථෛ', 'ffo': 'දෛ', 'ffO': 'ධෛ', 'ffk': 'නෛ', 'ff|': 'ඳෛ', 'ffm': 'පෛ', 'ffM': 'ඵෛ', 'ffn': 'බෛ', 'ffN': 'භෛ', 'ffu': 'මෛ', 'ffU': 'ඹෛ', 'ffh': 'යෛ', 'ffr': 'රෛ', 'ff,': 'ලෛ', 'ffj': 'වෛ', 'ffY': 'ශෛ', 'ffI': 'ෂෛ', 'ffi': 'සෛ', 'ffy': 'හෛ', 'ff<': 'ළෛ', 'ff¿': 'ළුෛ', 'ff*': 'ෆෛ', 'ff`.': 'ඟෛ', 'ff`P': 'ඦෛ', 'ff`v': 'ඬෛ', 'ff`o': 'ඳෛ'}


Need to remove 'ළුෛ' and add special cases "එෛ" and "ත්‍රෛ"

In [5]:
set_1["fft"] = "එෛ"
set_1["ff;%"] = "ත්‍රෛ"
del set_1["ff¿"]
char_set_list.append(set_1)

Even though "ප්‍රෞ" is the only commonly used combination, no harm in generating for other consonants.

In [6]:
set_2 = generator("f", "%!", "", "්‍රෞ")
printer(set_2)

{'fl%!': 'ක්‍රෞ', 'fL%!': 'ඛ්‍රෞ', 'f.%!': 'ග්‍රෞ', 'f>%!': 'ඝ්‍රෞ', 'fX%!': 'ඞ්‍රෞ', 'fÕ%!': 'ඟ්‍රෞ', 'fp%!': 'ච්‍රෞ', 'fP%!': 'ඡ්‍රෞ', 'fc%!': 'ජ්‍රෞ', 'fCO%!': 'ඣ්‍රෞ', 'f®%!': 'ඣ්‍රෞ', 'f[%!': 'ඤ්‍රෞ', 'f{%!': 'ඥ්‍රෞ', 'fg%!': 'ට්‍රෞ', 'fG%!': 'ඨ්‍රෞ', 'fv%!': 'ඩ්‍රෞ', 'fV%!': 'ඪ්‍රෞ', 'fK%!': 'ණ්‍රෞ', 'f~%!': 'ඬ්‍රෞ', 'f;%!': 'ත්‍රෞ', 'f:%!': 'ථ්‍රෞ', 'fo%!': 'ද්‍රෞ', 'fO%!': 'ධ්‍රෞ', 'fk%!': 'න්‍රෞ', 'f|%!': 'ඳ්‍රෞ', 'fm%!': 'ප්‍රෞ', 'fM%!': 'ඵ්‍රෞ', 'fn%!': 'බ්‍රෞ', 'fN%!': 'භ්‍රෞ', 'fu%!': 'ම්‍රෞ', 'fU%!': 'ඹ්‍රෞ', 'fh%!': 'ය්‍රෞ', 'fr%!': 'ර්‍රෞ', 'f,%!': 'ල්‍රෞ', 'fj%!': 'ව්‍රෞ', 'fY%!': 'ශ්‍රෞ', 'fI%!': 'ෂ්‍රෞ', 'fi%!': 'ස්‍රෞ', 'fy%!': 'හ්‍රෞ', 'f<%!': 'ළ්‍රෞ', 'f¿%!': 'ළු්‍රෞ', 'f*%!': 'ෆ්‍රෞ', 'f`.%!': 'ඟ්‍රෞ', 'f`P%!': 'ඦ්‍රෞ', 'f`v%!': 'ඬ්‍රෞ', 'f`o%!': 'ඳ්‍රෞ', }

Need to remove obviosly incorrect ones: 2 ඣ්‍රෞ, ඤ්‍රෞ, ඥ්‍රෞ, ණ්‍රෞ, ර්‍රෞ, ල්‍රෞ, ළු්‍රෞ

In [7]:
del set_2["fCO%!"]
del set_2["f®%!"]
del set_2["f[%!"]
del set_2["f{%!"]
del set_2["fK%!"]
del set_2["fr%!"]
del set_2["f,%!"]
del set_2["f¿%!"]
char_set_list.append(set_2)

Set 3 is '්‍යෝ' eg. 'ෂ්‍යෝ'.

In [8]:
set_3 = generator("f", "Hda", "", "්‍යෝ")
printer(set_3)

{'flHda': 'ක්‍යෝ', 'fLHda': 'ඛ්‍යෝ', 'f.Hda': 'ග්‍යෝ', 'f>Hda': 'ඝ්‍යෝ', 'fXHda': 'ඞ්‍යෝ', 'fÕHda': 'ඟ්‍යෝ', 'fpHda': 'ච්‍යෝ', 'fPHda': 'ඡ්‍යෝ', 'fcHda': 'ජ්‍යෝ', 'fCOHda': 'ඣ්‍යෝ', 'f®Hda': 'ඣ්‍යෝ', 'f[Hda': 'ඤ්‍යෝ', 'f{Hda': 'ඥ්‍යෝ', 'fgHda': 'ට්‍යෝ', 'fGHda': 'ඨ්‍යෝ', 'fvHda': 'ඩ්‍යෝ', 'fVHda': 'ඪ්‍යෝ', 'fKHda': 'ණ්‍යෝ', 'f~Hda': 'ඬ්‍යෝ', 'f;Hda': 'ත්‍යෝ', 'f:Hda': 'ථ්‍යෝ', 'foHda': 'ද්‍යෝ', 'fOHda': 'ධ්‍යෝ', 'fkHda': 'න්‍යෝ', 'f|Hda': 'ඳ්‍යෝ', 'fmHda': 'ප්‍යෝ', 'fMHda': 'ඵ්‍යෝ', 'fnHda': 'බ්‍යෝ', 'fNHda': 'භ්‍යෝ', 'fuHda': 'ම්‍යෝ', 'fUHda': 'ඹ්‍යෝ', 'fhHda': 'ය්‍යෝ', 'frHda': 'ර්‍යෝ', 'f,Hda': 'ල්‍යෝ', 'fjHda': 'ව්‍යෝ', 'fYHda': 'ශ්‍යෝ', 'fIHda': 'ෂ්‍යෝ', 'fiHda': 'ස්‍යෝ', 'fyHda': 'හ්‍යෝ', 'f<Hda': 'ළ්‍යෝ', 'f¿Hda': 'ළු්‍යෝ', 'f*Hda': 'ෆ්‍යෝ', 'f`.Hda': 'ඟ්‍යෝ', 'f`PHda': 'ඦ්‍යෝ', 'f`vHda': 'ඬ්‍යෝ', 'f`oHda': 'ඳ්‍යෝ', }

Need to remove ළු්‍යෝ

In [9]:
del set_3["f¿Hda"]
char_set_list.append(set_3)

Set 4 is '්‍යො' eg. 'ෂ්‍යො'.

In [10]:
set_4 = generator("f", "Hd", "", "්‍යො")
printer(set_4)

{'flHd': 'ක්‍යො', 'fLHd': 'ඛ්‍යො', 'f.Hd': 'ග්‍යො', 'f>Hd': 'ඝ්‍යො', 'fXHd': 'ඞ්‍යො', 'fÕHd': 'ඟ්‍යො', 'fpHd': 'ච්‍යො', 'fPHd': 'ඡ්‍යො', 'fcHd': 'ජ්‍යො', 'fCOHd': 'ඣ්‍යො', 'f®Hd': 'ඣ්‍යො', 'f[Hd': 'ඤ්‍යො', 'f{Hd': 'ඥ්‍යො', 'fgHd': 'ට්‍යො', 'fGHd': 'ඨ්‍යො', 'fvHd': 'ඩ්‍යො', 'fVHd': 'ඪ්‍යො', 'fKHd': 'ණ්‍යො', 'f~Hd': 'ඬ්‍යො', 'f;Hd': 'ත්‍යො', 'f:Hd': 'ථ්‍යො', 'foHd': 'ද්‍යො', 'fOHd': 'ධ්‍යො', 'fkHd': 'න්‍යො', 'f|Hd': 'ඳ්‍යො', 'fmHd': 'ප්‍යො', 'fMHd': 'ඵ්‍යො', 'fnHd': 'බ්‍යො', 'fNHd': 'භ්‍යො', 'fuHd': 'ම්‍යො', 'fUHd': 'ඹ්‍යො', 'fhHd': 'ය්‍යො', 'frHd': 'ර්‍යො', 'f,Hd': 'ල්‍යො', 'fjHd': 'ව්‍යො', 'fYHd': 'ශ්‍යො', 'fIHd': 'ෂ්‍යො', 'fiHd': 'ස්‍යො', 'fyHd': 'හ්‍යො', 'f<Hd': 'ළ්‍යො', 'f¿Hd': 'ළු්‍යො', 'f*Hd': 'ෆ්‍යො', 'f`.Hd': 'ඟ්‍යො', 'f`PHd': 'ඦ්‍යො', 'f`vHd': 'ඬ්‍යො', 'f`oHd': 'ඳ්‍යො', }

Need to remove ළු්‍යො.

In [11]:
del set_4["f¿Hd"]
char_set_list.append(set_4)

Set 5 is '්‍යෙ' eg. 'ෂ්‍යෙ'.

In [12]:
set_5 = generator("f", "H", "", "්‍යෙ")
printer(set_5)

{'flH': 'ක්‍යෙ', 'fLH': 'ඛ්‍යෙ', 'f.H': 'ග්‍යෙ', 'f>H': 'ඝ්‍යෙ', 'fXH': 'ඞ්‍යෙ', 'fÕH': 'ඟ්‍යෙ', 'fpH': 'ච්‍යෙ', 'fPH': 'ඡ්‍යෙ', 'fcH': 'ජ්‍යෙ', 'fCOH': 'ඣ්‍යෙ', 'f®H': 'ඣ්‍යෙ', 'f[H': 'ඤ්‍යෙ', 'f{H': 'ඥ්‍යෙ', 'fgH': 'ට්‍යෙ', 'fGH': 'ඨ්‍යෙ', 'fvH': 'ඩ්‍යෙ', 'fVH': 'ඪ්‍යෙ', 'fKH': 'ණ්‍යෙ', 'f~H': 'ඬ්‍යෙ', 'f;H': 'ත්‍යෙ', 'f:H': 'ථ්‍යෙ', 'foH': 'ද්‍යෙ', 'fOH': 'ධ්‍යෙ', 'fkH': 'න්‍යෙ', 'f|H': 'ඳ්‍යෙ', 'fmH': 'ප්‍යෙ', 'fMH': 'ඵ්‍යෙ', 'fnH': 'බ්‍යෙ', 'fNH': 'භ්‍යෙ', 'fuH': 'ම්‍යෙ', 'fUH': 'ඹ්‍යෙ', 'fhH': 'ය්‍යෙ', 'frH': 'ර්‍යෙ', 'f,H': 'ල්‍යෙ', 'fjH': 'ව්‍යෙ', 'fYH': 'ශ්‍යෙ', 'fIH': 'ෂ්‍යෙ', 'fiH': 'ස්‍යෙ', 'fyH': 'හ්‍යෙ', 'f<H': 'ළ්‍යෙ', 'f¿H': 'ළු්‍යෙ', 'f*H': 'ෆ්‍යෙ', 'f`.H': 'ඟ්‍යෙ', 'f`PH': 'ඦ්‍යෙ', 'f`vH': 'ඬ්‍යෙ', 'f`oH': 'ඳ්‍යෙ', }

Need to remove ළු්‍යෙ. And adding special case "hH_": "ර්‍ය්‍ය"

In [13]:
set_5["hH_"] = "ර්‍ය්‍ය"
del set_5["f¿H"]
char_set_list.append(set_5)